In [13]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
#Load Datasets

## load Project Sunroof Data
sr_data = pd.read_csv("data/project-sunroof-census_tract.csv", index_col="region_name")
sr_data = sr_data.rename_axis(index={"region_name": "GEOID"})
sr_data.index = sr_data.index.astype(int)
#print(sr_data.head(5))

#load Energy Equity Project Data
eep_data = pd.read_csv("data/eep_final_data.csv", index_col="GEOID")
eep_data.index = eep_data.index.astype(int)
#print(eep_data.head(5))

#load Justice40 Data
j40_data = pd.read_csv("data/1.0-communities.csv", index_col="Census tract 2010 ID")
j40_data = j40_data.rename(columns={"Census tract 2010 ID": "GEOID"})
j40_data.index = j40_data.index.astype(int)
#print(j40_data.head(5))

#load Opportunity Zone Data
oz_data = pd.read_csv("data/Opportunity_Zone_ExportTable1.csv", index_col="Tract_Boundaries_ID")
oz_data = oz_data.rename(columns={"Tract_Boundaries_ID": "GEOID"})
oz_data.index = oz_data.index.astype(int)
#print(oz_data.head(5))

/var/folders/67/r87g720s2653g554zdhtz1vh0000gn/T/ipykernel_11230/4072438724.py:15: DtypeWarning: Columns (18,26,70,72,85,131) have mixed types. Specify dtype option on import or set low_memory=False.
  j40_data = pd.read_csv("data/1.0-communities.csv", index_col="Census tract 2010 ID")


                     OID_  Tract_Boundaries_NAME Tract_Boundaries_STATE_NAME   
Tract_Boundaries_ID                                                            
1001020100              1             10010201.0                     Alabama  \
1001020200              2             10010202.0                     Alabama   
1001020300              3             10010203.0                     Alabama   
1001020400              4             10010204.0                     Alabama   
1001020500              5             10010205.0                     Alabama   

                    Tract_Boundaries_ST_ABBREV   
Tract_Boundaries_ID                              
1001020100                                  AL  \
1001020200                                  AL   
1001020300                                  AL   
1001020400                                  AL   
1001020500                                  AL   

                                         Poverty_csv_Geography   
Tract_Boundaries_ID   

In [ ]:
#Check datasets for shape
print(sr_data.shape)
print(eep_data.shape)
print(j40_data.shape)
print(sr_geodata.shape) #Missing about 10,000 wHY??
print(eep_geodata.shape)

In [9]:
#Combine data
sr_selected_cols = sr_data[['percent_covered', 'number_of_panels_total', 'number_of_panels_median', 'kw_median', 'kw_total', 
                            'yearly_sunlight_kwh_median', 'yearly_sunlight_kwh_total', 'carbon_offset_metric_tons', 'existing_installs_count']]

eep_selected_cols = eep_data[['STATE_NAME', 'energy_burden', 'num_solar_installers', 'median_income_solar', 'total_pop', 'year_built', 
                              'total_households', 'median_income', 'owner_occupied', 'renter_occupied', 'bipoc_percent', 'community_power_score',  ]]

j40_selected_cols = j40_data[['Total population', 'Is low income?']]

merged_data = pd.merge(sr_selected_cols, eep_selected_cols, left_index=True, right_index=True)
merged_data = pd.merge(merged_data, j40_selected_cols, left_index=True, right_index=True )

print(merged_data.head(5))

            percent_covered  number_of_panels_total  number_of_panels_median   
1001020100       103.360215                 39080.0                     48.0  \
1001020200       104.472843                 59054.0                     44.0   
1001020300       101.278409                 80364.0                     41.0   
1001020400       101.384451                 94598.0                     54.0   
1001020600       113.555195                 93919.0                     48.0   

            kw_median  kw_total  yearly_sunlight_kwh_median   
1001020100      12.00   9770.00                 14854.40286  \
1001020200      11.00  14763.50                 13564.14519   
1001020300      10.25  20091.00                 12725.93064   
1001020400      13.50  23649.50                 16586.32144   
1001020600      12.00  23479.75                 15360.34045   

            yearly_sunlight_kwh_total  carbon_offset_metric_tons   
1001020100                 12300000.0                7325.568357  \
1001

In [11]:
#Clean Data
cleaned_data = merged_data.copy()

#TODO
#EEP energy_burden has 100% values for some tracts
#SR has negative values

TypeError: '<' not supported between instances of 'str' and 'int'